## Data 

In [1]:
import numpy as np
import h5py
import numpy as np
import tensorflow as tf
from keras.losses import categorical_crossentropy
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc

Using TensorFlow backend.


In [2]:
f = h5py.File("data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z",'r')
treeArray = f['t_allpar_new'][()]

In [3]:
import itertools
x = [len(list(y)) for _,y in itertools.groupby(treeArray['j_index'])]
j_cIndex = np.array([],dtype='int8')

for i in x:
    new_jet_index = np.arange(i)
    j_cIndex = np.append(j_cIndex, new_jet_index)

In [4]:
split_index = np.array([])
for i in range(len(x)):
    if i%2470 == 0:
        split_index = np.append(split_index,np.sum(x[:i]))
split_index = np.append(split_index,np.sum(x))
split_index = split_index.astype(int)

In [21]:
ds_index = treeArray['index']
ds_JCPT = treeArray['j1_pt']
ds_JCETA = treeArray['j1_eta']
ds_JCPHI = treeArray['j1_phi']
ds_JCMASS = treeArray['j_mass']
ds_Nconstituents = treeArray['j_multiplicity']
ds_constituents_index = treeArray['j_index']
ds_JCDELTAETA = treeArray['j1_etarot']
ds_JCDELTAPHI = treeArray['j1_phirot']
ds_PT = treeArray['j_pt']
ds_ETA = treeArray['j1_etarel']
ds_PHI = treeArray['j1_phirel']

ds_label = np.vstack((treeArray['j_g'],treeArray['j_q'],treeArray['j_w'],treeArray['j_z'],treeArray['j_t'])).T

In [23]:
Label = ['index', 'JCPT', 'JCETA', 'JCPHI', 'JCMASS', 'Nconstituents', 'constituents_index', 'JCDELTAETA', 'JCDELTAPHI', 'PT', 'ETA','PHI','label']
for i in range(len(split_index)-1):
    with h5py.File('data/Particle_%d.h5'%i) as f2:
        for lb in Label:
            f2.create_dataset(lb, data=eval('ds_%s'%lb)[split_index[i]:split_index[i+1]])

D:\Softwares\Anacoda\envs\tf2\lib\site-packages\ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Model

In [29]:
##------------calculate jet constituents energy, if you already had this information don't use it.
def jce_np(jcpt, jceta, jcmass):
    jcp = jcpt*np.cosh(jceta)
    return (jcp**2+jcmass**2)**0.5


#-------- attach integers to your keys  this example is : 'index' -> 0-th data, 'JCPT' ->1-th data
# and 'index' is event index
# 'JCPT' is constituents transvers momentum (PT)
# 'JCETA' is constituents ETA
# 'JCPHI' is constituents angle around beam axis
# 'Ncontituents' is Number of constituents in the jet which the constitunt belong
# 'constiteunts_index' is constituents index in each jet ('constiteunts' is typo)
# 'JCDELTAETA' is constituents ETA related to jet axis
# 'JCDELTAPHI' is constituents ETA related to jet axis
# 'PT' is transvers momentum (PT) of jet which which the constitunt belong
# 'ETA' is ETA of jet which which the constitunt belong
# 'PHi' is PHI of jet which which the constitunt belong


Labels = ['index', 'JCPT', 'JCETA', 'JCPHI', 'JCMASS', 'Nconstituents', 'constituents_index', 'JCDELTAETA', 'JCDELTAPHI', 'PT', 'ETA', 'PHI']
_index, _jcpt, _jceta, _jcphi, _jcmass, _Ncons, _consindex, _jcdelteta, _jcdeltphi, _jpt, _jeta, _jphi = (i for i in range(len(Labels)))

##--------------------------------------------------h5py file to ParticleNet input structure
def h5_to_data(h5path):
    Data = {'mask':[], 'points':[], 'features':[]}
    f = h5py.File(h5path,'r')
    fc = np.array([f[lb][()] for lb in Labels])
    fc = fc.transpose((1,0))
    j0 = fc[0][_index]
    
    JCE = jce_np(fc[:,_jcpt], fc[:,_jceta], fc[:,_jcphi])
    logpt = np.log(fc[:,_jcpt])
    loge = np.log(JCE)
    relatpt = fc[:,_jcpt]/fc[:,_jpt]
    mask, features, points = np.zeros((100,1)), np.zeros((100,5)), np.zeros((100,2)) # prepare constituents list
    Nfc = len(fc)
    for j in range(len(fc)):
        if fc[j][_Ncons]>100:
            if j< Nfc-1:
                j0 = fc[j+1][_index]
            continue
        if fc[j][_index]!=j0:
            j0 = fc[j][_index]
            Data['mask'].append(mask)
            Data['points'].append(points)
            Data['features'].append(features)
            mask, features, points = np.zeros((100,1)), np.zeros((100,5)), np.zeros((100,2)) # prepare constituents list
            continue
        jc = int(fc[j][_consindex])
#         jce = JCE[j]
        
        points[jc] = np.array([fc[j][_jcdelteta], fc[j][_jcdeltphi] ])
        mask[jc] = logpt[j]
        features[jc] = np.array([logpt[j], loge[j], fc[j][_jcdelteta], fc[j][_jcdeltphi], relatpt[j]])
    return Data

##==========================================================================================
##----------merging 2 sample with ParticleNet input structure
def merging(gg,qq):
    total={}
    total['mask']=gg["mask"]+qq["mask"]
    total['features']=gg["features"]+qq["features"]
    total['points']=gg['points']+qq['points']
    return total
##--------- seperate inputs to training, validation and testing with ratio you give
def separatedata(features_list,y,rateval,ratetest):
    features_train, features_test, features_val={},{},{}
    from sklearn.model_selection import train_test_split
    mask = features_list["mask"]
    features = features_list["features"]
    points = features_list["points"]
    X_ind = [i for i in range(len(y))]
    X_train, X_ind, y_train, y_ind = train_test_split(X_ind, y, test_size=rateval+ratetest)
    N=int(len(X_ind)*rateval/(rateval+ratetest))
    X_val, X_test = X_ind[:N], X_ind[N:]
    y_val, y_test = y_ind[:N], y_ind[N:]
    features_train['mask']=np.array([mask[i] for i in X_train])
    features_train['features']=np.array([features[i] for i in X_train])
    features_train['points']=np.array([points[i] for i in X_train])
    
    features_test['mask']=np.array([mask[i] for i in X_test])
    features_test['features']=np.array([features[i] for i in X_test])
    features_test['points']=np.array([points[i] for i in X_test])
    
    features_val['mask']=np.array([mask[i] for i in X_val])
    features_val['features']=np.array([features[i] for i in X_val])
    features_val['points']=np.array([points[i] for i in X_val])
    
    return features_train, features_val, features_test,np.array(y_train), np.array(y_val), np.array(y_test)

In [32]:
treeArray['j_index']

array([   101333,    101333,    101333, ..., 300123134, 300123134,
       300123134])

In [30]:
N = 40
Data = {'mask':[], 'points':[], 'features':[]}
for i in tqdm(range(N)):
    h5Path = "data/Particle_"+str(i)+".h5"
    data = h5_to_data(h5Path)
    Data = merging(Data,data)
print("check shape: ",Data['mask'][0].shape,Data['points'][0].shape,Data['features'][0].shape)

  0%|                                                                                           | 0/40 [00:00<?, ?it/s]


IndexError: index 101333 is out of bounds for axis 0 with size 100

In [ ]:
# ## binary calssification: 
# y=[[0,1] for i in range(len(Data['mask']))]+[[1,0] for i in range(len(Data2['mask']))] 
# total = merging(Data, Data2)

y = h5py.File(h5Path)['label'][()]

# del Data
X_train, X_val, X_test, y_train, y_val, y_test = separatedata(Data,y,0.25,0.25)

In [ ]:
## check the shape
print([X_train[i][0].shape for i in ['mask', 'points', 'features']])

In [ ]:
import sys
sys.path.insert(0,'models')
from tf_keras_model import get_particle_net_lite

In [ ]:
input_shapes={'points': X_train['points'][0].shape, 'features': X_train['features'][0].shape, 'mask': X_train['mask'][0].shape}
num_classes = 5
model = get_particle_net_lite(num_classes, input_shapes)

In [ ]:
batch_size = 1024
epochs = 3

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [16]:
model.fit(X_train ,y_train,
          batch_size=batch_size,
          epochs=epochs, 
          validation_data=(X_val, y_val),
          shuffle=True ,
#           callbacks=callbacks
         )
model.evaluate(X_test,  y_test, verbose=2)

Train on 63767 samples, validate on 31884 samples
Epoch 1/10
63767/63767 [==============================] - 416s 7ms/sample - loss: 1.5888 - accuracy: 0.2811 - val_loss: 1.5696 - val_accuracy: 0.2817
Epoch 2/10
63767/63767 [==============================] - 371s 6ms/sample - loss: 1.5664 - accuracy: 0.2810 - val_loss: 1.5672 - val_accuracy: 0.2817
Epoch 3/10
63767/63767 [==============================] - 368s 6ms/sample - loss: 1.5660 - accuracy: 0.2814 - val_loss: 1.5671 - val_accuracy: 0.2818
Epoch 4/10
63767/63767 [==============================] - 376s 6ms/sample - loss: 1.5659 - accuracy: 0.2811 - val_loss: 1.5673 - val_accuracy: 0.2817
Epoch 5/10
18432/63767 [=======>......................] - ETA: 4:26 - loss: 1.5638 - accuracy: 0.2861

KeyboardInterrupt: 

## Evaluation

In [ ]:
y_score = model.predict(X_test)[:,0]
test=[i[1] for i in y_test]
fpr , tpr , thresholds = roc_curve ( test , y_score)
roc_auc = auc(tpr,fpr )
print("The area under the curves are:")
print("AUC:{0:.9f}".format(roc_auc))
# FalsePositiveFull, TruePositiveFull, ThresholdFull = metrics.roc_curve(y_test,Predictions)
plt.figure(figsize=(8,16))
plt.subplot(2,1,1)

plt.plot(tpr,fpr, label='Fully supervised: AUC={0:.9f}'.format(roc_auc))
plt.ylabel('1-False Positive Rate',fontsize=20)
plt.xlabel('True Positive Rate',fontsize=20)
plt.plot([0, 1], [0, 1], 'k--')
plt.legend(bbox_to_anchor=(0.8, -0.17),ncol=2)
plt.tight_layout()
# plt.savefig("./H5/ParticleNet/Myneighnorhood_Particlenet_roc.png")
# hf = h5py.File("/home/ja2006203966/script/Network/data/H5/ParticleNet/PR.h5", 'w')
# hf.create_dataset('fpr', data=fpr)
# hf.create_dataset('tpr', data=tpr)
# hf.close()

plt.show()

In [ ]:
plt.figure(figsize=(8,8))
LOSS = pd.read_csv(save_dir)
# LOSS = pd.read_csv("/home/ja2006203966/script/Network/data/myparticlenet_training_log.csv")

plt.title("Learning Curve")
plt.plot(LOSS["loss"], label='loss',c='blue')
plt.plot(LOSS["val_loss"], label='val_loss',c='red')
plt.plot(LOSS["accuracy"], linestyle='--', label='accuracy',c='blue')
plt.plot(LOSS["val_accuracy"], linestyle='--', label='val_accuracy',c='red')
# plt.ylim([0.3,1])
plt.ylabel('loss',fontsize=20)
plt.xlabel('epoch',fontsize=20)
# plt.legend(bbox_to_anchor=(0.8, -0.17),ncol=2)
plt.legend(bbox_to_anchor=(0.8, -0.17),ncol=2)
plt.tight_layout()
plt.savefig("/home/ja2006203966/script/Network/data/H5/ParticleNet/ParticleNetloss.png")

plt.show()